In [26]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import resnet50
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

In [15]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)
class FaceNetModel(nn.Module):
    def __init__(self, pretrained=False):
        super(FaceNetModel, self).__init__()

        self.model = resnet50(pretrained)
        embedding_size = 128
        num_classes = 500
        self.cnn = nn.Sequential(
            self.model.conv1,
            self.model.bn1,
            self.model.relu,
            self.model.maxpool,
            self.model.layer1,
            self.model.layer2,
            self.model.layer3,
            self.model.layer4)

        # modify fc layer based on https://arxiv.org/abs/1703.07737
        self.model.fc = nn.Sequential(
            Flatten(),
            # nn.Linear(100352, 1024),
            # nn.BatchNorm1d(1024),
            # nn.ReLU(),
            nn.Linear(100352, embedding_size))

        self.model.classifier = nn.Linear(embedding_size, num_classes)

    def l2_norm(self, input):
        input_size = input.size()
        buffer = torch.pow(input, 2)
        normp = torch.sum(buffer, 1).add_(1e-10)
        norm = torch.sqrt(normp)
        _output = torch.div(input, norm.view(-1, 1).expand_as(input))
        output = _output.view(input_size)
        return output

    def freeze_all(self):
        for param in self.model.parameters():
            param.requires_grad = False

    def unfreeze_all(self):
        for param in self.model.parameters():
            param.requires_grad = True

    def freeze_fc(self):
        for param in self.model.fc.parameters():
            param.requires_grad = False

    def unfreeze_fc(self):
        for param in self.model.fc.parameters():
            param.requires_grad = True

    def freeze_only(self, freeze):
        for name, child in self.model.named_children():
            if name in freeze:
                for param in child.parameters():
                    param.requires_grad = False
            else:
                for param in child.parameters():
                    param.requires_grad = True

    def unfreeze_only(self, unfreeze):
        for name, child in self.model.named_children():
            if name in unfreeze:
                for param in child.parameters():
                    param.requires_grad = True
            else:
                for param in child.parameters():
                    param.requires_grad = False

    # returns face embedding(embedding_size)
    def forward(self, x):
        x = self.cnn(x)
        x = self.model.fc(x)

        features = self.l2_norm(x)
        # Multiply by alpha = 10 as suggested in https://arxiv.org/pdf/1703.09507.pdf
        alpha = 10
        features = features * alpha
        return features

    def forward_classifier(self, x):
        features = self.forward(x)
        res = self.model.classifier(features)
        return res

In [16]:
model = FaceNetModel()

In [84]:
path='../code/log/best_state_2.pth'
device = torch.device("cuda:0")
model = FaceNetModel()
model.to(device)
model.eval()
state = torch.load(path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])

<All keys matched successfully>

In [45]:
trfrm = transforms.Compose([
    lambda x: x.convert('RGB'),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])
topil = transforms.ToPILImage()
totensor = transforms.Compose(trfrm.transforms[:-1])

def get_distance(img1, img2,model):
    model.eval()
    with torch.no_grad():
        x1 = trfrm(img1).unsqueeze(0)
        x2 = trfrm(img2).unsqueeze(0)
        x1,x2 = x1.to('cuda:0'), x2.to('cuda:0')
        embed1 = model(x1)
        embed2 = model(x2)
        return F.pairwise_distance(embed1, embed2)

In [86]:
img1 = 'C:/Users/Gemmechu/Documents/files/UMich/DogOwner/code/datasets/dogOwner/7/1.jpg'
img2 = 'C:/Users/Gemmechu/Documents/files/UMich/DogOwner/code/datasets/dogOwner/7/2.jpg'
img3 = 'C:/Users/Gemmechu/Pictures/dog_and_owner/Newfolder/sarah.png'
img4 = 'C:/Users/Gemmechu/Pictures/dog_and_owner/Newfolder/sarahd.jpg'
imgA = Image.open(img3).convert('RGB')
imgB = Image.open(img4).convert('RGB')

In [87]:
dist = get_distance(imgA, imgB,model)
dist

tensor([0.9936], device='cuda:0')